In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import torch
print(torch.cuda.is_available())  # Returns True if GPU is available
print(torch.cuda.current_device())  # Displays the current GPU device
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # Displays GPU name


True
0
NVIDIA GeForce RTX 3050 Laptop GPU


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import load_dataset

import torch
import transformers
import pandas as pd
from tqdm import tqdm

# Initialize tqdm for pandas
tqdm.pandas(disable=True)


In [8]:
# Load tokenizer and model from Hugging Face
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
# Load IMDb dataset
dataset = load_dataset("imdb")

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [12]:
# Print the first 5 samples from the IMDb training dataset
for i in range(5):
    print(dataset["train"][i])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
# Evaluate before fine-tuning
metrics_before = trainer.evaluate(tokenized_datasets["test"].shuffle().select(range(500)))
print("Before Fine-tuning Accuracy:", metrics_before["eval_accuracy"])
    

In [13]:
# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16= True
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle().select(range(1000)),
    eval_dataset=tokenized_datasets["test"].shuffle().select(range(500)),
    compute_metrics=compute_metrics
)
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}





In [16]:
# Before fine-tuning
metrics_before = trainer.evaluate()
print("Before Fine-tuning Accuracy:", metrics_before["eval_accuracy"])

/home/mcqueen/Documents/gpu-powered-bert-finetuning/myenv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Before Fine-tuning Accuracy: 0.524


In [12]:
# Fine-tune the model
trainer.train()

/home/mcqueen/Documents/gpu-powered-bert-finetuning/myenv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,0.436442
2,No log,0.280218
3,No log,0.220967


/home/mcqueen/Documents/gpu-powered-bert-finetuning/myenv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=189, training_loss=0.34171989481285137, metrics={'train_runtime': 180.8239, 'train_samples_per_second': 16.591, 'train_steps_per_second': 1.045, 'total_flos': 789333166080000.0, 'train_loss': 0.34171989481285137, 'epoch': 3.0})

In [13]:
# Save the Fine-Tuned Model
model.save_pretrained("fine_tuned_bert")
tokenizer.save_pretrained("fine_tuned_bert")

('fine_tuned_bert/tokenizer_config.json',
 'fine_tuned_bert/special_tokens_map.json',
 'fine_tuned_bert/vocab.txt',
 'fine_tuned_bert/added_tokens.json',
 'fine_tuned_bert/tokenizer.json')

In [17]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained("fine_tuned_bert")
tokenizer = BertTokenizer.from_pretrained("fine_tuned_bert")


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_datasets["test"].shuffle().select(range(500)),
    compute_metrics=compute_metrics
)

# Evaluate
metrics = trainer.evaluate()
print("Fine-tuned Model Accuracy:", metrics["eval_accuracy"])


/home/mcqueen/Documents/gpu-powered-bert-finetuning/myenv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Fine-tuned Model Accuracy: 0.894


In [14]:
# Load the fine-tuned model
classifier = pipeline("text-classification", model="fine_tuned_bert", device=0 if torch.cuda.is_available() else -1)

Device set to use cuda:0


In [18]:
# Test with sample text
print(classifier("This movie is bad"))
print(classifier("This movie is GOOD"))

[{'label': 'LABEL_0', 'score': 0.8461263179779053}]
[{'label': 'LABEL_1', 'score': 0.9351804852485657}]


In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [25]:
sample = dataset["train"][0]["text"]
sample

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [22]:
import re

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    
    return text

In [26]:
clean_text(sample)

'i rented i am curiousyellow from my video store because of all the controversy that surrounded it when it was first released in  i also heard that at first it was seized by us customs if it ever tried to enter this country therefore being a fan of films considered controversial i really had to see this for myselfbr br the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such as the vietnam war and race issues in the united states in between asking politicians and ordinary denizens of stockholm about their opinions on politics she has sex with her drama teacher classmates and married menbr br what kills me about i am curiousyellow is that  years ago this was considered pornographic really the sex and nudity scenes are few and far between even then its not shot like some cheapl

In [ ]:
from nltk.corpus import stopwords